In [167]:
import math

class Node:
    def __init__(self, node, x, y, node_name):
        self.id = node
        self.x = x
        self.y = y
        self.node_name = node_name
        self.adj = {}
    
    def calculate_dist_to(self, neighbour):
        return (abs(neighbour.get_x() - self.x), abs(neighbour.get_y() - self.y))
    
    def add_neighbour(self, neighbour):
        self.adj[neighbour] = self.calculate_dist_to(neighbour)
    
    def get_id(self):
        return self.id
    
    def get_x(self):
        return self.x
    
    def get_y(self):
        return self.y
    
    def get_degree(self):
        return len(self.adj)
    
    def get_neighbours(self):
        return self.adj
    
    def __repr__(self):
        return str(self.id) + ' adjacent: ' + str([x.id for x in self.adj]) + '\n'
    
    def __str__(self):
        return self.__repr__()
    

In [168]:
# This class implements an undirected graph
class Graph:
    def __init__(self):
        self.node_map = {}
        self.num_nodes = 0
    
    def add_node(self, node_id, x, y, node_name):
        self.num_nodes = self.num_nodes + 1
        new_node = Node(node_id, x, y, node_name)
        self.node_map[node_id] = new_node
        return new_node
    
    def is_valid_node_id(self, node_id):
        if (node_id not in self.node_map):
            return False
        return True
    
    def add_edge(self, node_one, node_two):
        if (self.is_valid_node_id(node_one) and self.is_valid_node_id(node_two)):
            self.node_map[node_one].add_neighbour(self.node_map[node_two])
            self.node_map[node_two].add_neighbour(self.node_map[node_one])
    
    def get_node(self, node):
        return self.node_map[node]
    
    def get_degree(self, node):
        return self.node_map[node].get_degree()
    
    def get_num_nodes(self):
        return self.num_nodes
    
    def get_nodes(self):
        return self.node_map.values()
    
    def __repr__(self):
        result = ''
        for node in self.node_map.values():
            result += str(node)
        return result

In [169]:
import heapq

class Dijkstra:
    
    def __init__(self, graph, source):
        num_nodes = graph.get_num_nodes()
        self.source_id = source
        self.dist_to = {}  # distance of shortest s->v path
        self.prev = {}  # previous node on shortest s->v path
        self.pq = [] # priority queue of vertices
        
        nodes = graph.get_nodes()
        
        for node in nodes:
            node_id = node.get_id()
            self.dist_to[node_id] = math.inf
            self.prev[node_id] = None
        self.dist_to[source] = 0.0
        
        # relax vertices in order of distance from s
        heapq.heappush(self.pq, (source, self.dist_to[source]))
        while(len(self.pq)):
            v = heapq.heappop(self.pq)[0]
            for neighbour in graph.get_node(v).get_neighbours():
                weight = graph.get_node(v).calculate_dist_to(neighbour)
                weight = weight[0] + weight[1]
                self.__relax(v, neighbour.get_id(), weight)
                
    def __relax(self, v_id, w_id, weight):            
            # get the index, since we start from 1, we need to minus one
            if (self.dist_to[w_id] > self.dist_to[v_id] + weight):
                self.dist_to[w_id] = self.dist_to[v_id] + weight
                self.prev[w_id] = v_id
                heapq.heappush(self.pq, (w_id, self.dist_to[w_id]))
                
    def dist_to_node(self, v):
        return self.dist_to[v]
    
    def get_path(self, target_id):
        path = []
        current_node_id = target_id
        while (current_node_id != self.source_id):
            path.append(current_node_id)
            current_node_id = self.prev[current_node_id]
        path.append(current_node_id)
        return path[::-1]

In [170]:
import requests
import json

r = requests.get("http://showmyway.comp.nus.edu.sg/getMapInfo.php?Building=COM1&Level=2")

body = r.json()

g = Graph()

for node in body["map"]:
    node_id = int(node['nodeId'])
    x = int(node['x'])
    y = int(node['y'])
    node_name = node['nodeName']
    link_to = node['linkTo'].split(', ')
    
    g.add_node(node_id, x, y, node_name)
    
    for i in range(len(link_to)):
        g.add_edge(node_id, int(link_to[i]))

print("Graph Created Successfully:")
print(g)

print("Enter source node: ")
source = int(input())
shortest_path = Dijkstra(g, source)

print()

print("Enter destination node: ")
destination = int(input())

print()

distance = shortest_path.dist_to_node(destination)

print("Shortest path from " + str(source) + " to " + str(destination) + ": ")
print(shortest_path.get_path(destination))

print()
print("Shortest distance is " + str(distance / 100) + "m")

Graph Created Successfully:
1 adjacent: [2]
2 adjacent: [3, 1, 4]
3 adjacent: [2]
4 adjacent: [7, 2, 6, 5]
5 adjacent: [8, 4]
6 adjacent: [4]
7 adjacent: [10, 4]
8 adjacent: [9, 10, 5]
9 adjacent: [8]
10 adjacent: [7, 11, 8]
11 adjacent: [12, 10, 14, 13]
12 adjacent: [11]
13 adjacent: [11, 36]
14 adjacent: [15, 11, 37]
15 adjacent: [32, 14]
16 adjacent: [18, 37]
17 adjacent: [21, 39, 19]
18 adjacent: [22, 20, 16]
19 adjacent: [17]
20 adjacent: [18]
21 adjacent: [24, 23, 17]
22 adjacent: [25, 34, 18]
23 adjacent: [21]
24 adjacent: [21, 28, 27]
25 adjacent: [22]
26 adjacent: [28, 29, 34]
27 adjacent: [24]
28 adjacent: [24, 26, 30]
29 adjacent: [26, 31]
30 adjacent: [28]
31 adjacent: [29]
32 adjacent: [15, 39, 33]
33 adjacent: [32]
34 adjacent: [22, 35, 26]
35 adjacent: [34]
36 adjacent: [13]
37 adjacent: [14, 38, 16]
38 adjacent: [37]
39 adjacent: [32, 40, 17]
40 adjacent: [39]

Enter source node: 
1

Enter destination node: 
28

Shortest path from 1 to 28: 
[1, 2, 4, 7, 10, 11, 14, 37, 